## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


100%|██████████| 1/1 [00:07<00:00,  7.26s/it]

Number of chunks =  31
due to the prominent spurious correlation problem (Geirhos

et al. 2018; Hu et al. 2022), i.e., non-causal correlations

learned from the limited training annotations. The trained

model is supposed to only exploit AU-specific features

representing the corresponding facial appearance changes,

which is much harder to achieve with non-negligible spuri-

ous correlation. First, the model aims to estimate the inten-

sity values for multiple AUs simultaneously, and thus AU

co-occurrences may lead to spurious correlation. As in Fig. 1

(b), the model is supposed to capture both facial appearance

changes from the cheek region for AU6 and lip corner re-

gion for AU12. However, their co-occurrence patterns (e.g.,

both AUs with the same intensity) that frequently occur in

keyframe annotations may mislead the model into learning

features of only one of them (Shi et al. 2022), e.g., using fea-

tures from the lip corner region to represent both AUs and

ignoring tho

In [3]:
import importlib.util
import sys
import os

# 添加项目路径到sys.path
project_path = 'D:/知识图谱与认知智能/大作业/knowledge_graph-main/knowledge_graph-main'
if project_path not in sys.path:
    sys.path.append(project_path)

# 加载 df_helpers 模块
module_path = os.path.join(project_path, 'helpers', 'df_helpers.py')
spec = importlib.util.spec_from_file_location("helpers.df_helpers", module_path)
df_helpers = importlib.util.module_from_spec(spec)
spec.loader.exec_module(df_helpers)

# 使用模块中的函数
#documents2Dataframe = df_helpers.documents2Dataframe
print("Module imported successfully")


Module imported successfully


## Create a dataframe of all the chunks

In [4]:

from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(31, 3)


,text,source,chunk_id
0,Trend-Aware Supervision: On Learning Invarianc...,data_input\cureus\5.txt,3898818987224bd68c5f7adc0ae75fbd
1,lem by raising trend awareness during training...,data_input\cureus\5.txt,a2124b6d251740ea8f21798f8bdcd795
2,"†Corresponding author.Copyright © 2024, Associ...",data_input\cureus\5.txt,3d4a933a2a284dff9100dd33f051422d
3,due to the prominent spurious correlation prob...,data_input\cureus\5.txt,be2f5f07257b46bb96e6805d8057cebd
4,instead of AU features to represent the intens...,data_input\cureus\5.txt,30c4fc379a664f6190d98a8ee6643cb3


## Extract Concepts

In [5]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [6]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='gpt-4')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"3_graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"3_chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

2


c:\Users\社会主义接班人\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




ERROR ### Here is the buggy response:  [
   {
       "node_1": "image",
       "node_2": "segment",
       "edge": "Images are contained in each segment"
   },
   {
       "node_1": "intensity label",
       "node_2": "AU",
       "edge": "Intensity labels are given for the AU in the first and the last images"
   },
   {
       "node_1": "intensity label",
       "node_2": "normalized range",
       "edge": "Intensity labels are uniformly normalized from range [0, 5] to range [0, 1]"
   },
   {
       "node_1": "goal",
       "node_2": "optimal Θ",
       "edge": "Goal is to learn an optimal Θ for an image-based AU intensity estimation model"
   },
   {
       "node_1": "batch",
       "node_2": "pre-processed segments",
       "edge": "A batch of pre-processed segments is taken as input in the training stage"
   },
   {
       "node_1": "image",
       "node_2": "backbone network",
       "edge": "Each image is fed into a backbone network to extract global feature"
   },
   {
      

,node_1,node_2,edge,chunk_id,count
0,trend-aware supervision,semi-supervised au intensity estimation,Trend-Aware Supervision (TAS) is proposed as a...,3898818987224bd68c5f7adc0ae75fbd,4
1,trend-aware supervision,keyframe annotations,Trend-Aware Supervision (TAS) utilizes trend i...,3898818987224bd68c5f7adc0ae75fbd,4
2,trend-aware supervision,au-specific features,Trend-Aware Supervision (TAS) aims to learn AU...,3898818987224bd68c5f7adc0ae75fbd,4
3,semi-supervised au intensity estimation,keyframe annotations,Semi-supervised AU intensity estimation uses o...,3898818987224bd68c5f7adc0ae75fbd,4
4,semi-supervised au intensity estimation,spurious correlation problem,The lack of annotations in semi-supervised AU ...,3898818987224bd68c5f7adc0ae75fbd,4


## Calculating contextual proximity

In [7]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
6325,∆fc,facial appearance,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",2,contextual proximity
6326,∆fc,frame,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",4,contextual proximity
6327,∆fc,intra-trend ranking awareness,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",6,contextual proximity
6328,∆fc,intra-trend speed awareness,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",2,contextual proximity
6329,∆fc,loss function,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",2,contextual proximity


### Merge both the dataframes

In [8]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,16,method,"45889a5b81fa426bb6d03ab275b2a811,45889a5b81fa4...",contextual proximity,2
1,16,sgd,"45889a5b81fa426bb6d03ab275b2a811,45889a5b81fa4...",contextual proximity,2
2,16,t,"45889a5b81fa426bb6d03ab275b2a811,45889a5b81fa4...",contextual proximity,3
3,16,λrank,"45889a5b81fa426bb6d03ab275b2a811,45889a5b81fa4...",contextual proximity,2
4,16,λspd,"45889a5b81fa426bb6d03ab275b2a811,45889a5b81fa4...",contextual proximity,2
...,...,...,...,...,...
2990,∆fc,facial appearance,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",The relationship between ∆fc and the facial ap...,6
2991,∆fc,frame,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",contextual proximity,4
2992,∆fc,intra-trend ranking awareness,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",contextual proximity,6
2993,∆fc,intra-trend speed awareness,"cd8ba14b35f54a6cb9f7e8c5dd696468,cd8ba14b35f54...",contextual proximity,2


## Calculate the NetworkX Graph

In [9]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(313,)

In [10]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [11]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  19
[['16', 'adult subjects', 'annotations', 'automatic differentiation', 'batch size', 'bp4d', 'datasets', 'disfa', 'dlib', 'facial images', 'facs-quantified au intensity labels', 'fera 2015 challenge', 'fully-supervised methods', 'intra-class correlation (icc(3,1))', 'learning rate', 'mean absolute error (mae)', 'method', 'nvidia a100 tensor core gpu', 'paszke', 'pytorch', 'segmenti, j', 'semi-supervised au inten- sity estimation methods', 'sgd', 't', 't, k', 't, α, λrank, λspd', 'weight decay', 'α', 'λrank', 'λspd', 'λsub'], ['6th international conference on learning representations', 'asian conference on computer vision', 'bp4d-spontaneous: a high-resolution spontaneous 3d dynamic facial expression database', 'context-aware feature and label fusion for facial action unit intensity estimation with partially labeled data', 'deep facial action unit recognition and intensity estimation', 'ieee conference on computer vision and pattern recognition', 'ieee transac

### Create a dataframe for community colors

In [12]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,16,#57dba4,1
1,adult subjects,#57dba4,1
2,annotations,#57dba4,1
3,automatic differentiation,#57dba4,1
4,batch size,#57dba4,1
...,...,...,...
308,semi-supervised methods,#5795db,17
309,ladder network,#db57cc,18
310,semi-supervised learning,#db57cc,18
311,multi-task generalization,#dbb257,19


### Add colors to the graph

In [13]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [14]:
from pyvis.network import Network

graph_output_directory = "./docs/3_index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])
html_content = net.generate_html()

# Write the HTML content to a file with UTF-8 encoding
with open(graph_output_directory, 'w', encoding='utf-8') as f:
    f.write(html_content)
# net.show(graph_output_directory, notebook=False)

In [15]:
# import pandas as pd

# # 保存节点信息
# nodes_data = []
# for node in G.nodes(data=True):
#     nodes_data.append({
#         'node': node[0],
#         'group': node[1].get('group'),
#         'color': node[1].get('color'),
#         'size': node[1].get('size')
#     })

# df_nodes = pd.DataFrame(nodes_data)
# df_nodes.to_csv("nodes_data.csv", index=False)

# # 保存边信息
# edges_data = []
# for edge in G.edges(data=True):
#     edges_data.append({
#         'node_1': edge[0],
#         'node_2': edge[1],
#         'weight': edge[2].get('weight')
#     })

# df_edges = pd.DataFrame(edges_data)
# df_edges.to_csv("edges_data.csv", index=False)
# communities_data = []
# for group, community in enumerate(communities, 1):
#     for node in community:
#         communities_data.append({'node': node, 'group': group})

# df_communities = pd.DataFrame(communities_data)
# df_communities.to_csv("communities_data.csv", index=False)
